In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/Churn_Modelling.csv')

In [3]:
df.shape

(10000, 14)

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [10]:
df['Gender'].replace(['Male', 'Female'],[1, 0], inplace= True)
df['Gender'].unique()

array([0, 1])

In [11]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [12]:
X=df.drop(['Exited'],axis=1)
y=df['Exited']

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.10,random_state=5)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.10,random_state=5)
print("X_train size is {}".format(X_train.shape[0]))
print("X_val size is {}".format(X_val.shape[0]))
print("X_test size is {}".format(X_test.shape[0]))

X_train size is 8100
X_val size is 900
X_test size is 1000


In [14]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
num_cols=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
num_subset=scaler.fit_transform(X_train[num_cols])
X_train_num_df=pd.DataFrame(num_subset,columns=num_cols)
X_train_num_df['Geography']=list(X_train['Geography'])
X_train_num_df['Gender']=list(X_train['Gender'])
X_train_num_df['HasCrCard']=list(X_train['HasCrCard'])
X_train_num_df['IsActiveMember']=list(X_train['IsActiveMember'])
X_train_num_df.head()
## Standardise the Validation data
num_subset=scaler.fit_transform(X_val[num_cols])
X_val_num_df=pd.DataFrame(num_subset,columns=num_cols)
X_val_num_df['Geography']=list(X_val['Geography'])
X_val_num_df['Gender']=list(X_val['Gender'])
X_val_num_df['HasCrCard']=list(X_val['HasCrCard'])
X_val_num_df['IsActiveMember']=list(X_val['IsActiveMember'])
## Standardise the Test data
num_subset=scaler.fit_transform(X_test[num_cols])
X_test_num_df=pd.DataFrame(num_subset,columns=num_cols)
X_test_num_df['Geography']=list(X_test['Geography'])
X_test_num_df['Gender']=list(X_test['Gender'])
X_test_num_df['HasCrCard']=list(X_test['HasCrCard'])
X_test_num_df['IsActiveMember']=list(X_test['IsActiveMember'])

In [15]:
## Convert the categorical features to numerical
X_train_num_df=pd.get_dummies(X_train_num_df,columns=['Geography','Gender'])
X_test_num_df=pd.get_dummies(X_test_num_df,columns=['Geography','Gender'])
X_val_num_df=pd.get_dummies(X_val_num_df,columns=['Geography','Gender'])
X_train_num_df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,HasCrCard,IsActiveMember,Geography_France,Geography_Germany,Geography_Spain,Gender_0,Gender_1
0,-1.178587,-1.041960,-1.732257,0.198686,0.820905,1.560315,1,1,1,0,0,0,1
1,-0.380169,-1.326982,1.730718,-0.022020,-0.907991,-0.713592,1,0,0,0,1,1,0
2,-0.349062,1.808258,-0.693364,0.681178,0.820905,-1.126515,1,0,0,1,0,1,0
3,0.625629,2.378302,-0.347067,-1.229191,0.820905,-1.682740,1,1,1,0,0,0,1
4,-0.203895,-1.136967,1.730718,0.924256,-0.907991,1.332535,1,1,0,0,1,0,1


In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model=Sequential()
model.add(Dense(7,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [17]:
import tensorflow as tf
optimizer=tf.keras.optimizers.Adam(0.01)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [18]:
model.fit(X_train_num_df,y_train,epochs=100,batch_size=10,verbose=1)

Epoch 1/100
810/810 [==============================] - 2s 2ms/step - loss: 0.3863 - accuracy: 0.8347
Epoch 2/100
810/810 [==============================] - 2s 2ms/step - loss: 0.3489 - accuracy: 0.8578
Epoch 3/100
810/810 [==============================] - 2s 3ms/step - loss: 0.3477 - accuracy: 0.8553
Epoch 4/100
810/810 [==============================] - 3s 3ms/step - loss: 0.3413 - accuracy: 0.8594
Epoch 5/100
810/810 [==============================] - 2s 3ms/step - loss: 0.3425 - accuracy: 0.8610
Epoch 6/100
810/810 [==============================] - 2s 2ms/step - loss: 0.3391 - accuracy: 0.8626
Epoch 7/100
810/810 [==============================] - 1s 2ms/step - loss: 0.3382 - accuracy: 0.8637
Epoch 8/100
810/810 [==============================] - 1s 2ms/step - loss: 0.3379 - accuracy: 0.8636
Epoch 9/100
810/810 [==============================] - 1s 2ms/step - loss: 0.3378 - accuracy: 0.8623
Epoch 10/100
810/810 [==============================] - 1s 2ms/step - loss: 0.3349 - accura

In [19]:
y_pred_val=model.predict(X_val_num_df)
y_pred_val[y_pred_val>0.5]=1
y_pred_val[y_pred_val <0.5]=0

29/29 [==============================] - 0s 1ms/step


In [20]:
y_pred_val=y_pred_val.tolist()
X_compare_val=X_val.copy()
X_compare_val['y_actual']=y_val
X_compare_val['y_pred']=y_pred_val
X_compare_val.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,y_actual,y_pred
340,341,15745844,Kerr,642,Germany,0,40,6,129502.49,2,0,1,86099.23,1,[0.0]
8622,8623,15747772,Cunningham,706,Germany,1,36,9,58571.18,2,1,0,40774.01,0,[0.0]
8401,8402,15663134,Uspenskaya,535,Spain,1,58,1,0.00,2,1,1,11779.98,1,[0.0]
4338,4339,15798439,Davidson,714,Spain,1,25,2,0.00,1,1,1,132979.43,0,[0.0]
8915,8916,15804962,Nnaife,606,France,1,36,1,155655.46,1,1,1,192387.51,1,[0.0]
2624,2625,15788946,Anthony,605,Spain,0,29,3,116805.82,1,0,0,4092.75,0,[0.0]
2234,2235,15713655,Calabrese,720,France,0,38,10,0.00,2,1,1,56229.72,1,[0.0]
349,350,15576256,Yusupova,582,France,1,39,5,0.00,2,1,1,129892.93,0,[0.0]
3719,3720,15577999,Sleeman,850,France,0,62,1,124678.35,1,1,0,70916.00,1,[1.0]
2171,2172,15747174,Hao,526,Germany,1,58,9,190298.89,2,1,1,191263.76,0,[0.0]


In [21]:
from sklearn.metrics import confusion_matrix
cm_val=confusion_matrix(y_val,y_pred_val)
cm_val

array([[689,  27],
       [101,  83]])